In [6]:
import sys
sys.path.append('..')
import os, csv, json, zipfile, requests
from io import BytesIO
from collections import defaultdict

In [5]:
del sys.modules['tools.constants']
del sys.modules['tools.instruments']

KeyError: 'tools.instruments'

In [7]:
from tools.constants import DATADIR, SRF_CONTRACTS_CSV_PATH, SRF_METADATA_CSV_PATH,\
                              SRF_CONTRACTS_JSON_PATH
from tools.instruments import instruments

..\tools\instruments.py:253: UserWarning: SRF_CONTRACTS_JSON_PATH 파일이 존재하지 않습니다.
  


### 0. SRF 홈페이지에서 meta 데이터 다운로드 후 파일로 저장

In [48]:
#url
url = "https://data.nasdaq.com/api/v3/databases/SRF/metadata"

In [49]:
print("다운로드 시작...")
req = requests.get(url)
print("다운로드 완료")

다운로드 시작...
다운로드 완료


In [50]:
#파일로 저장
file = zipfile.ZipFile(BytesIO(req.content))
temp_dir = os.path.join(DATADIR,'nasdaq-data-link', 'temp')
file.extractall(temp_dir)
filename = next(os.walk(temp_dir), (None, None, []))[2][0]  # [] if no file
filepath = os.path.join(temp_dir, filename)
os.replace(filepath, SRF_METADATA_CSV_PATH)
os.rmdir(temp_dir)

### 1.meta.csv 파일로부터 월물 정보 정리 후 저장

In [18]:
contracts = []
with open(SRF_METADATA_CSV_PATH, encoding='utf-8', mode='r') as f:
    lines = csv.DictReader(f)
    for line in lines:
        srf = line['code'][:-5]
        if len(instruments.filter(srf=srf)) == 0:
            #print(line['code'], line['name'])
            continue
        symbol = instruments.filter(srf=srf)[0].symbol
        row = (symbol, srf, line['code'], line['from_date'], line['to_date'], line['refreshed_at'][:10], line['name'])
        contracts.append(row)


In [9]:
# meta 파일에 들어있는 월물 리스트를 필요한 상품만 골라 월물시간순으로 정리
def sort_contracts(row):
    symbol = row[0]
    year = int(row[2][-4:])
    month = instruments.month_code(row[2][-5])
    return (symbol, year, month)
    

In [19]:
contracts.sort(key=sort_contracts)

* 확인하기 편하게 csv 파일로 저장

In [20]:
fields = ['symbol', 'srf', 'contract', 'from_date', 'to_date', 'refreshed_at', 'name']
with open(SRF_CONTRACTS_CSV_PATH, 'w', newline='') as file:
    wr = csv.writer(file)
    wr.writerow(fields)
    wr.writerows(contracts)

* 사용시 속도 향상을 위해 json으로 저장

In [21]:
from collections import defaultdict
import json

In [22]:
contracts_dict = defaultdict(list)
for contract in contracts:
    contracts_dict[contract[0]].append(contract[:-1])

In [23]:
with open(SRF_CONTRACTS_JSON_PATH, 'w') as f:
    json.dump(contracts_dict, f)

### 2. 저장된 데이터 확인

In [24]:
with open(SRF_CONTRACTS_CSV_PATH, encoding='utf-8', mode='r') as f:
    lines = csv.DictReader(f)
    for line in lines:
        print(line)

{'symbol': 'AD', 'srf': 'CME_AD', 'contract': 'CME_ADH1987', 'from_date': '1987-01-13', 'to_date': '1987-03-16', 'refreshed_at': '2015-04-27', 'name': 'CME Australian Dollar AUD, March 1987 (ADH1987)'}
{'symbol': 'AD', 'srf': 'CME_AD', 'contract': 'CME_ADM1987', 'from_date': '1987-01-13', 'to_date': '1987-06-15', 'refreshed_at': '2015-04-27', 'name': 'CME Australian Dollar AUD, June 1987 (ADM1987)'}
{'symbol': 'AD', 'srf': 'CME_AD', 'contract': 'CME_ADU1987', 'from_date': '1987-01-16', 'to_date': '1987-09-14', 'refreshed_at': '2015-04-27', 'name': 'CME Australian Dollar AUD, September 1987 (ADU1987)'}
{'symbol': 'AD', 'srf': 'CME_AD', 'contract': 'CME_ADZ1987', 'from_date': '1987-03-18', 'to_date': '1987-12-14', 'refreshed_at': '2015-04-27', 'name': 'CME Australian Dollar AUD, December 1987 (ADZ1987)'}
{'symbol': 'AD', 'srf': 'CME_AD', 'contract': 'CME_ADH1988', 'from_date': '1987-06-17', 'to_date': '1988-03-14', 'refreshed_at': '2015-04-27', 'name': 'CME Australian Dollar AUD, March 1

{'symbol': 'EC', 'srf': 'CME_EC', 'contract': 'CME_ECM2001', 'from_date': '2000-05-18', 'to_date': '2001-06-18', 'refreshed_at': '2015-05-21', 'name': 'CME Euro FX, June 2001 (ECM2001)'}
{'symbol': 'EC', 'srf': 'CME_EC', 'contract': 'CME_ECU2001', 'from_date': '2000-09-08', 'to_date': '2001-09-17', 'refreshed_at': '2015-05-21', 'name': 'CME Euro FX, September 2001 (ECU2001)'}
{'symbol': 'EC', 'srf': 'CME_EC', 'contract': 'CME_ECZ2001', 'from_date': '2000-11-22', 'to_date': '2001-12-17', 'refreshed_at': '2015-05-21', 'name': 'CME Euro FX, December 2001 (ECZ2001)'}
{'symbol': 'EC', 'srf': 'CME_EC', 'contract': 'CME_ECH2002', 'from_date': '2000-12-18', 'to_date': '2002-03-18', 'refreshed_at': '2015-05-21', 'name': 'CME Euro FX, March 2002 (ECH2002)'}
{'symbol': 'EC', 'srf': 'CME_EC', 'contract': 'CME_ECM2002', 'from_date': '2001-03-20', 'to_date': '2002-06-17', 'refreshed_at': '2015-05-21', 'name': 'CME Euro FX, June 2002 (ECM2002)'}
{'symbol': 'EC', 'srf': 'CME_EC', 'contract': 'CME_ECU2

{'symbol': 'HO', 'srf': 'CME_HO', 'contract': 'CME_HOH1998', 'from_date': '1996-09-11', 'to_date': '1998-02-27', 'refreshed_at': '2015-05-21', 'name': 'NYMEX Heating Oil, March 1998 (HOH1998)'}
{'symbol': 'HO', 'srf': 'CME_HO', 'contract': 'CME_HOJ1998', 'from_date': '1996-10-07', 'to_date': '1998-03-31', 'refreshed_at': '2015-05-21', 'name': 'NYMEX Heating Oil, April 1998 (HOJ1998)'}
{'symbol': 'HO', 'srf': 'CME_HO', 'contract': 'CME_HOK1998', 'from_date': '1996-11-01', 'to_date': '1998-04-30', 'refreshed_at': '2015-05-21', 'name': 'NYMEX Heating Oil, May 1998 (HOK1998)'}
{'symbol': 'HO', 'srf': 'CME_HO', 'contract': 'CME_HOM1998', 'from_date': '1996-12-02', 'to_date': '1998-05-29', 'refreshed_at': '2015-05-21', 'name': 'NYMEX Heating Oil, June 1998 (HOM1998)'}
{'symbol': 'HO', 'srf': 'CME_HO', 'contract': 'CME_HON1998', 'from_date': '1997-01-06', 'to_date': '1998-06-30', 'refreshed_at': '2015-05-21', 'name': 'NYMEX Heating Oil, July 1998 (HON1998)'}
{'symbol': 'HO', 'srf': 'CME_HO', 

{'symbol': 'OJ', 'srf': 'ICE_OJ', 'contract': 'ICE_OJF1979', 'from_date': '1977-09-14', 'to_date': '1979-01-18', 'refreshed_at': '2015-05-21', 'name': 'ICE Orange Juice, January 1979 (OJF1979)'}
{'symbol': 'OJ', 'srf': 'ICE_OJ', 'contract': 'ICE_OJH1979', 'from_date': '1977-10-12', 'to_date': '1979-03-19', 'refreshed_at': '2015-05-21', 'name': 'ICE Orange Juice, March 1979 (OJH1979)'}
{'symbol': 'OJ', 'srf': 'ICE_OJ', 'contract': 'ICE_OJK1979', 'from_date': '1977-12-02', 'to_date': '1979-05-17', 'refreshed_at': '2015-05-21', 'name': 'ICE Orange Juice, May 1979 (OJK1979)'}
{'symbol': 'OJ', 'srf': 'ICE_OJ', 'contract': 'ICE_OJN1979', 'from_date': '1978-02-02', 'to_date': '1979-07-18', 'refreshed_at': '2015-05-21', 'name': 'ICE Orange Juice, July 1979 (OJN1979)'}
{'symbol': 'OJ', 'srf': 'ICE_OJ', 'contract': 'ICE_OJU1979', 'from_date': '1978-06-30', 'to_date': '1979-09-17', 'refreshed_at': '2015-05-21', 'name': 'ICE Orange Juice, September 1979 (OJU1979)'}
{'symbol': 'OJ', 'srf': 'ICE_OJ'

{'symbol': 'TU', 'srf': 'CME_TU', 'contract': 'CME_TUU2005', 'from_date': '2005-04-08', 'to_date': '2005-09-30', 'refreshed_at': '2015-05-21', 'name': 'CBOT 2-year US Treasury Note, September 2005 (TUU2005)'}
{'symbol': 'TU', 'srf': 'CME_TU', 'contract': 'CME_TUZ2005', 'from_date': '2005-07-20', 'to_date': '2005-12-30', 'refreshed_at': '2015-05-21', 'name': 'CBOT 2-year US Treasury Note, December 2005 (TUZ2005)'}
{'symbol': 'TU', 'srf': 'CME_TU', 'contract': 'CME_TUH2006', 'from_date': '2005-11-03', 'to_date': '2006-03-31', 'refreshed_at': '2015-05-21', 'name': 'CBOT 2-year US Treasury Note, March 2006 (TUH2006)'}
{'symbol': 'TU', 'srf': 'CME_TU', 'contract': 'CME_TUM2006', 'from_date': '2006-01-12', 'to_date': '2006-06-30', 'refreshed_at': '2015-05-21', 'name': 'CBOT 2-year US Treasury Note, June 2006 (TUM2006)'}
{'symbol': 'TU', 'srf': 'CME_TU', 'contract': 'CME_TUU2006', 'from_date': '2006-04-03', 'to_date': '2006-09-29', 'refreshed_at': '2015-05-21', 'name': 'CBOT 2-year US Treasur

In [25]:
with open(SRF_CONTRACTS_JSON_PATH, 'r') as f:
    contracts = json.load(f)

In [26]:
contracts

{'AD': [['AD',
   'CME_AD',
   'CME_ADH1987',
   '1987-01-13',
   '1987-03-16',
   '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADM1987', '1987-01-13', '1987-06-15', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADU1987', '1987-01-16', '1987-09-14', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADZ1987', '1987-03-18', '1987-12-14', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADH1988', '1987-06-17', '1988-03-14', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADM1988', '1987-10-30', '1988-06-13', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADU1988', '1988-02-24', '1988-09-19', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADZ1988', '1988-05-17', '1988-12-19', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADH1989', '1988-09-02', '1989-03-13', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADM1989', '1989-01-03', '1989-06-19', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADU1989', '1989-02-21', '1989-09-18', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADZ1989', '1989-04-19', '1989-12-18', '2015-04-27'],
  ['AD', 'CME_AD', 'CME_ADH1990', '1989-06-20', '1990-0